# Projeto

## Data Preparation

- Eliminar Colunas IDentificadoras e tornar ID como PK

- Eliminar Erros de Input
    - Format "Age" & Check SSN pattern
    - Negative Numeric Variables: 'Age', 'Num_Bank_Accounts', 'NumofLoan'
    - Age Restriction (GREATER than 0 && LESS than 150)
<br><br>
- Eliminar 9 Records de 'Monthly_Balance' igual a -3.33333E+26

- Trabalhar com 'Credit_History_Age' como variavel numerica 

- Variable Enconding of 'Age' and 'Credit_History_Age'

In [1]:
import pandas as pd
import re
import numpy as np
import dslabs_functions as dslab

file_tag = 'credit_score'
filename = 'class_credit_score.csv'
original_dataset: pd.DataFrame = pd.read_csv(filename, index_col='ID', sep=',', decimal='.', na_values='')

# Drop IDentifying Columns
dataset_cleaned = original_dataset.drop(columns=['Customer_ID', 'Name', 'SSN'], errors='ignore')

# Erase Input Errors
## Check and Clean 'SSN' & 'Age' Column
pattern = r'^\d{3}-\d{2}-\d{4}$'  # Replace this with your SSN regex pattern
#dataset_cleaned['SSN'] = dataset_cleaned['SSN'].apply(lambda x: x if re.match(pattern, str(x)) else np.nan)
dataset_cleaned['Age'] = dataset_cleaned['Age'].astype(str).str.replace('_', '', regex=False)

## Transfrom 'Age' from SIMBOLIC to NUMERIC
dataset_cleaned['Age'] = pd.to_numeric(dataset_cleaned['Age'], errors='coerce')

## Invalid Numeric Variables 'Age', 'Num_Bank_Accounts', 'NumofLoan' Column
dataset_cleaned['Age'] = dataset_cleaned['Age'].apply(lambda x: np.nan if x == -500 or x >= 150 else x)
dataset_cleaned['Num_Bank_Accounts'] = dataset_cleaned['Num_Bank_Accounts'].apply(lambda x: x if x >= 0 else np.nan)
dataset_cleaned['NumofLoan'] = dataset_cleaned['NumofLoan'].apply(lambda x: x if x >= 0 else np.nan)

# Monthly_Balance 
dataset_cleaned['MonthlyBalance'] = dataset_cleaned['MonthlyBalance'].apply(lambda x: x if x >= 0 else np.nan)

## Transform 'Credit_History_Age' from SIMBOLIC to NUMERIC
hist_age = dataset_cleaned["Credit_History_Age"]
hist_vals = []

for val in hist_age:
    matches = re.findall(r'(\d+) Years and (\d+) Months', str(val))
    if not matches:  # Use `not matches` instead of `matches == []`
        hist_vals.append(np.nan)
    else:
        hist_vals.append(float(matches[0][0]) + (float(matches[0][1]) / 12))

dataset_cleaned["Credit_History_Age"] = hist_vals

### Drop 'Credit_History_Age' as OBJECT 
object_columns = dataset_cleaned.select_dtypes(include=['object']).columns.tolist()
if "CreditHistoryAge" in object_columns:
    dataset_cleaned.drop(columns=["CreditHistoryAge"], inplace=True)

# Get Variables Type Right (Object to Category)
variables_types: dict[str, list] = dslab.get_variable_types(dataset_cleaned)
numeric: list[str] = variables_types["numeric"]
symbolic: list[str] = variables_types["symbolic"]
binary: list[str] = variables_types["binary"]

print(variables_types)

# Object to Category (same as symbolic in Pandas)
dataset_cleaned[symbolic] = dataset_cleaned[symbolic].apply(lambda x: x.astype("category"))
print(dataset_cleaned.dtypes)

{'numeric': ['Age', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'NumofLoan', 'Delay_from_due_date', 'NumofDelayedPayment', 'ChangedCreditLimit', 'NumCreditInquiries', 'OutstandingDebt', 'CreditUtilizationRatio', 'Credit_History_Age', 'TotalEMIpermonth', 'Amountinvestedmonthly', 'MonthlyBalance'], 'binary': ['Credit_Score'], 'date': [], 'symbolic': ['Month', 'Occupation', 'Type_of_Loan', 'CreditMix', 'Payment_of_Min_Amount', 'Payment_Behaviour']}
Month                     category
Age                        float64
Occupation                category
Annual_Income              float64
Monthly_Inhand_Salary      float64
Num_Bank_Accounts          float64
Num_Credit_Card              int64
Interest_Rate                int64
NumofLoan                  float64
Type_of_Loan              category
Delay_from_due_date          int64
NumofDelayedPayment        float64
ChangedCreditLimit         float64
NumCreditInquiries         float64
Credi

### Variable Enconding

- *'Credit_History_Age' feito na 1ª célula*

1. Order Enconding Binary *(order is irrelevant)*
    - 'Credit_Score' <br><br>

2. Order Enconding *(order is relevant)*
    - 'CreditMix' [Bad, Standard, Good] (3)<br><br>
    - 'Payment_of_Min_Amount' ['No', 'NM', 'Yes'] (3) <br><br>
    - 'Payment_Behaviour' ['Low_spent_Small_value_payments', 'Low_spent_Medium_value_payments', 'Low_spent_Large_value_payments', 'High_spent_Small_value_payments', 'High_spent_Medium_value_payments', 'High_spent_Large_value_payments'] (6) <br><br>

3. Cyclic Variables
    - 'Month' ordenar primeiro [January, February, March, ...]<br><br>

4. TODO
    - 'Occupation'<br><br>
    - 'Type_of_Loan'

In [2]:
# Collect Individual values for each symbolic var, in order to encode the rest
dataset_cleaned

for v in variables_types["symbolic"]:
    print(v, dataset_cleaned[v].unique())

Month ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August']
Categories (8, object): ['April', 'August', 'February', 'January', 'July', 'June', 'March', 'May']
Occupation ['Scientist', NaN, 'Teacher', 'Engineer', 'Entrepreneur', ..., 'Accountant', 'Musician', 'Mechanic', 'Writer', 'Architect']
Length: 16
Categories (15, object): ['Accountant', 'Architect', 'Developer', 'Doctor', ..., 'Musician', 'Scientist', 'Teacher', 'Writer']
Type_of_Loan ['Auto Loan, Credit-Builder Loan, Personal Loan..., 'Credit-Builder Loan', 'Auto Loan, Auto Loan, and Not Specified', 'Not Specified', NaN, ..., 'Auto Loan, Payday Loan, Auto Loan, Student Lo..., 'Home Equity Loan, Payday Loan, Not Specified,..., 'Home Equity Loan, Auto Loan, Auto Loan, and A..., 'Payday Loan, Student Loan, Mortgage Loan, and..., 'Personal Loan, Auto Loan, Mortgage Loan, Stud...]
Length: 6261
Categories (6260, object): ['Auto Loan', 'Auto Loan, Auto Loan, Auto Loan, Auto Loan, C..., 'Auto Loan, Auto Loan, Auto L

In [6]:
# Enconding 'Data_Score' Binary - Ordinal Enconding (easiest one)
variables_types

yes_no: dict[str, int] = {"no": 0, "No": 0, "yes": 1, "Yes": 1}
credit_score_values: dict[str, int] = {"Poor": 0, "Good": 1}

encoding_binary: dict[str, dict[str, int]] = {
    "Credit_Score": credit_score_values,
}

data_encoded_binary: pd.DataFrame = dataset_cleaned.replace(encoding_binary, inplace=False)

# Enconding 'Credit_Mix' Non-Binary - Ordinal Enconding
credit_mix_values: dict[str, int] = {"Bad": 0, "Standard": 1, "Good": 2}

# Enconding 'Payment_of_Min_Amount' Non-Binary - Ordinal Enconding
payment_of_min_amount_values: dict[str, int] = {"No": 0, "NM": 1, "Yes": 2}

# Enconding 'Payment_Behaviour' Non-Binary - Ordinal Enconding
payment_behaviour_values: dict[str, int] = {"Low_spent_Small_value_payments": 0, "Low_spent_Medium_value_payments": 1, "Low_spent_Large_value_payments": 2,
                                             "High_spent_Small_value_payments": 3, "High_spent_Medium_value_payments": 4, "High_spent_Large_value_payments": 5}

encoding_non_binary: dict[str, dict[str, int]] = {
    "CreditMix": credit_mix_values,
    "Payment_of_Min_Amount": payment_of_min_amount_values,
    "Payment_Behaviour": payment_behaviour_values,
}

data_encoded_non_binary: pd.DataFrame = data_encoded_binary.replace(encoding_non_binary, inplace=False)
data_encoded_non_binary.head(1)

,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,NumofLoan,Type_of_Loan,...,CreditMix,OutstandingDebt,CreditUtilizationRatio,Credit_History_Age,Payment_of_Min_Amount,TotalEMIpermonth,Amountinvestedmonthly,Payment_Behaviour,MonthlyBalance,Credit_Score
ID,,,,,,,,,,,,,,,,,,,,,
0x1602,January,23.0,Scientist,19114.12,1824.843333,3.0,4,3,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,NaN,809.98,26.822620,22.083333,0,49.574949,80.415295,3,312.494089,1
0x1603,February,23.0,Scientist,19114.12,NaN,3.0,4,3,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,2,809.98,31.944960,NaN,0,49.574949,118.280222,2,284.629163,1
0x1604,March,NaN,Scientist,19114.12,NaN,3.0,4,3,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,2,809.98,28.609352,22.250000,0,49.574949,81.699521,1,331.209863,1
0x1605,April,23.0,Scientist,19114.12,NaN,3.0,4,3,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,2,809.98,31.377862,22.333333,0,49.574949,199.458074,0,223.451310,1
0x1606,May,23.0,Scientist,19114.12,1824.843333,3.0,4,3,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,2,809.98,24.797347,22.416667,0,49.574949,41.420153,4,341.489231,1
0x1607,June,23.0,Scientist,19114.12,NaN,3.0,4,3,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,2,809.98,27.262259,22.500000,0,49.574949,62.430172,NaN,340.479212,1
0x1608,July,23.0,Scientist,19114.12,1824.843333,3.0,4,3,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,2,809.98,22.537593,22.583333,0,49.574949,178.344067,0,244.565317,1
0x1609,August,23.0,Scientist,19114.12,1824.843333,3.0,4,3,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,2,809.98,23.933795,NaN,0,49.574949,24.785217,4,358.124168,1
0x160e,January,28.0,NaN,34847.84,3037.986667,2.0,4,6,1.0,Credit-Builder Loan,...,2,605.03,24.464031,26.583333,0,18.816215,104.291825,0,470.690627,1
